## 4. Using Fuzzywuzzy to get best matched dashboards

FuzzyWuzzy is a library of Python which is used for string matching. Fuzzy string matching is the process of finding strings that match a given pattern. Basically it uses <b>Levenshtein Distance</b> to calculate the differences between sequences.



In [1]:
#Logic to get the dashboards

def fuzzywuzzy_suggestion(corpus, input_query):
    for i in [corpus]:
        return process.extract(input_query, i, limit=10)

def dashboard_names_suggestion(master, suggestions, dashboard_names_list,metric_names_list):
    dashboard_names = []   
    for i in suggestions:
        
        if i in dashboard_names_list:    
            dashboard_names.append(i)
        
        elif i in metric_names_list:    
            dashboard_names = dashboard_names + (master.loc[(master.metric_names.str.lower() == i)].dashboard_names).to_list()
        
        else:
            dashboard_names = dashboard_names + (master.loc[(master.descriptor_names.str.lower() == i)].dashboard_names).to_list()

    return dashboard_names

In [2]:
token_set_ratio_sugg = {}

def fuzzywuzzy_scorers_suggestions(usersearch, corpus):
    from fuzzywuzzy import fuzz
    import pandas as pd
    from pandas.core.common import flatten
    import numpy as np

    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process

    import rapidfuzz
    from rapidfuzz import process, utils


    import warnings
    warnings.filterwarnings('ignore')

    
    for token in corpus:
        sugg_score = fuzz.token_set_ratio(usersearch, token)
        token_set_ratio_sugg[token] = sugg_score

        
def fuzzywuzzy_scorers_similiarity(scorers,suggestions_count,sort_scorers):
    
    import pandas as pd
    from pandas.core.common import flatten
    import numpy as np

    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process

    import rapidfuzz
    from rapidfuzz import process, utils


    import warnings
    warnings.filterwarnings('ignore') 
    
    
    scoreDf = pd.DataFrame()
    scoreDf = scoreDf.from_dict([token_set_ratio_sugg]).T.reset_index()
    scoreDf.columns = ['suggestion','token_set_ratio_sugg']
    
    return scoreDf[scorers].sort_values(by = sort_scorers, ascending = False).head(suggestions_count)

In [3]:
def init(input_query, mastercorpus, master,dashboard_names_list,metric_names_list):
    from fuzzywuzzy import fuzz
    import pandas as pd
    from pandas.core.common import flatten
    import numpy as np

    from fuzzywuzzy import fuzz
    from fuzzywuzzy import process

    import rapidfuzz
    from rapidfuzz import process, utils

    import warnings
    warnings.filterwarnings('ignore')
    import time
    
# FUNCTION CALLING 
    print('User Search : ' + str(input_query))
  
    fuzzywuzzy_scorers_suggestions(input_query.lower(), mastercorpus)

    fuzzyScorers = ['suggestion','token_set_ratio_sugg']
    scorersSorting = ['token_set_ratio_sugg']

    allScorersOutput = []


    fuzzywuzzySearchOutput = []

    # FUNCTION CALLING 
    scoreSortedDf = fuzzywuzzy_scorers_similiarity(scorers=fuzzyScorers, suggestions_count = 15,sort_scorers= scorersSorting)

    scoreSortedDf.sort_values(scorersSorting, ascending = False)

    #FUNCTION CALLING
    fuzzywuzzyDashboardsList = list(dashboard_names_suggestion(master, scoreSortedDf['suggestion'][:10],dashboard_names_list,metric_names_list))

    fuzzywuzzySearchOutput = list(dict.fromkeys(fuzzywuzzyDashboardsList))
    allScorersOutput.append(fuzzywuzzySearchOutput[:10])


    scorersSuggestionsDf = pd.concat([d.reset_index(drop=True) for d in [
                            scoreSortedDf.sort_values(['token_set_ratio_sugg'], ascending = False)[['suggestion']],
                           ]],axis=1)

    scorersSuggestionsDf.columns  = ['token_set_ratio_sugg']
    
    
    top_metric_list = scoreSortedDf['suggestion'][:10].to_list()
    top_dashboard_list = fuzzywuzzyDashboardsList[:10]

    
    master_new = master[master['dashboard_names'].isin(top_dashboard_list)]
   
    master_new = master_new[master_new['descriptor_names'].isin(top_metric_list)]


    finalSuggestionDF = master_new.groupby("dashboard_names").agg(list).reset_index()
    
    finalSuggestionDF["metric_names"] = finalSuggestionDF["metric_names"].apply(lambda x: list(set(x)))

    pd.set_option('display.max_colwidth', None)

    display(finalSuggestionDF)
        
        